In [ ]:
import cv2
import dlib
import imutils
from imutils import face_utils
from scipy.spatial import distance as dist
from google.colab import drive
from google.colab.patches import cv2_imshow # 導入顯示OpenCV格式影像套件包



In [ ]:
def eye_aspect_ratio(eye):
    p2_minus_p6 = dist.euclidean(eye[1], eye[5])
    p3_minus_p5 = dist.euclidean(eye[2], eye[4])
    p1_minus_p4 = dist.euclidean(eye[0], eye[3])
    ear = (p2_minus_p6 + p3_minus_p5) / (2.0 * p1_minus_p4)
    return ear

In [ ]:

FACIAL_LANDMARK_PREDICTOR = "/content/drive/MyDrive/碩一/EdgeAI/eye_predictor.dat"  
MINIMUM_EAR = 0.25
MAXIMUM_FRAME_COUNT = 30
EYE_CLOSED_COUNTER = 0

In [ ]:
faceDetector = dlib.get_frontal_face_detector()
landmarkFinder = dlib.shape_predictor(FACIAL_LANDMARK_PREDICTOR)

In [ ]:
(leftEyeStart, leftEyeEnd) = face_utils.FACIAL_LANDMARKS_IDXS["left_eye"]
(rightEyeStart, rightEyeEnd) = face_utils.FACIAL_LANDMARKS_IDXS["right_eye"]

In [ ]:
from IPython.display import HTML # 導入IPython.display HTML函式庫
from base64 import b64encode # 導入base64 baseencode函式庫
import cv2 # 導入OpenCV函式庫

v_in = '/content/drive/MyDrive/碩一/EdgeAI/prerecorded_video.mp4' # 指定輸入視頻名稱
v_out = '/content/drive/MyDrive/碩一/EdgeAI/prerecorded_video_out.mp4' # 指定輸出視頻名稱
v_avc = '/content/drive/MyDrive/碩一/EdgeAI/prerecorded_video_avc.mp4' # 指定輸出視頻名稱

cap = cv2.VideoCapture(v_in) # 讀取視頻檔案
video_w = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)) # 讀影視頻影格寬度
video_h = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT)) # 讀取視頻影格高度
video_c = int(cap.get(cv2.CAP_PROP_FRAME_COUNT)) # 讀取視頻影格數量
print (f'video width : {video_w}') # 列出影格寬度
print (f'video height : {video_h}') # 列出影格高度
print (f'frame count : {video_c}') # 列出影格數量

fourcc = cv2.VideoWriter_fourcc(*'XVID') # 指定輸出視頻格式，*'XVID' 或 *'MPV4' 皆可產出
writer = cv2.VideoWriter(v_out, fourcc, 30, (video_w, video_h)) # 指定寫入名稱、格式、速度(FPS)、尺寸

i = 0 # 清除執行影格數量
while(cap.isOpened()): # 若影片開啟正常則啟動循環
  ret, image = cap.read() # 讀取一個影格

  if ret == True: # 若影格讀取正常
    # image = imutils.resize(image, width=800)
    grayImage = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    faces = faceDetector(grayImage, 0)

    for face in faces:
        faceLandmarks = landmarkFinder(grayImage, face)
        faceLandmarks = face_utils.shape_to_np(faceLandmarks)

        leftEye = faceLandmarks[leftEyeStart:leftEyeEnd]
        rightEye = faceLandmarks[rightEyeStart:rightEyeEnd]

        leftEAR = eye_aspect_ratio(leftEye)
        rightEAR = eye_aspect_ratio(rightEye)

        ear = (leftEAR + rightEAR) / 2.0

        leftEyeHull = cv2.convexHull(leftEye)
        rightEyeHull = cv2.convexHull(rightEye)

        cv2.drawContours(image, [leftEyeHull], -1, (255, 0, 0), 2)
        cv2.drawContours(image, [rightEyeHull], -1, (255, 0, 0), 2)

        if ear < MINIMUM_EAR:
            EYE_CLOSED_COUNTER += 1
        else:
            EYE_CLOSED_COUNTER = 0

        image = cv2.putText(image, "EAR: {}".format(round(ear, 1)), (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)

        if EYE_CLOSED_COUNTER >= MAXIMUM_FRAME_COUNT:
            image = cv2.putText(image, "Drowsiness", (10, 50), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)

    writer.write(image)
    # # 在影格左上角加上一個'Output'黃色字樣
    # frame = cv2.putText(frame, "Output", (30,30), cv2.FONT_HERSHEY_COMPLEX, 1.0, (0,255,255), 1,cv2.LINE_AA)
    # writer.write(frame) # 將影格寫入輸出視頻檔案中
    i = i+1 # 執行影格數量加一
  else : # 若已讀不到影格
    print(f'count : {i}') # 列出總共執行影格數量
    break # 結束循環

cap.release() # 釋放影像擷取資源
writer.release() # 釋放影像寫入資源
print('done') # 列出結束訊息

# 以ffmpeg將mpv4格式的mp4檔案轉成一般品質avc格式的mp4檔案
!ffmpeg -i /content/drive/MyDrive/碩一/EdgeAI/prerecorded_video_out.mp4 -acodec libx264 /content/drive/MyDrive/碩一/EdgeAI/prerecorded_video_avc.mp4

vs_out = open(v_avc,'rb').read() # 開啟並讀取mp4格式視頻檔
data_url = "data:video/mp4;base64," + b64encode(vs_out).decode() # 設定顯示內容格式
HTML(f'<video width=400 controls><source src={data_url} type="video/mp4"></video>') # 將視頻顯示於視窗上

video width : 1280
video height : 720
frame count : 369
count : 369
done
ffmpeg version 3.4.8-0ubuntu0.2 Copyright (c) 2000-2020 the FFmpeg developers
  built with gcc 7 (Ubuntu 7.5.0-3ubuntu1~18.04)
  configuration: --prefix=/usr --extra-version=0ubuntu0.2 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --enable-gpl --disable-stripping --enable-avresample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librubberband --enable-librsvg --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvorbis --enable-libvpx --enable-libwavpack --enabl